In [7]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

In [8]:
dict_rxn_to_gene["METLEUex"]

['6520.1', '8140.1']

In [9]:
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/Non_masked_edges_node_names_explainer_subgraphs.csv").drop("Unnamed: 0",axis=1)


edges_node_names_explainer_subgraphs.query("node1 == 'METLEUex'")

,node1,node2,0,1,2,3,4,5,6,7,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
577,METLEUex,leu_L_c,0.017892,0.018656,0.012535,0.046060,0.012905,0.025395,0.013120,0.012988,...,0.023753,0.034653,0.046350,0.075669,0.035507,0.035372,0.008325,0.130646,0.025754,0.032150
578,METLEUex,leu_L_e,0.015301,0.018108,0.012095,0.039167,0.012481,0.036362,0.012547,0.012397,...,0.043190,0.033277,0.034498,0.037429,0.032685,0.043219,0.018571,0.036918,0.040904,0.038387


In [10]:

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == edges_node_names_explainer_subgraphs.shape[0]


2022-10-23 17:50:38,486	INFO worker.py:1518 -- Started a local Ray instance.


In [11]:
results_for_R = pd.concat([genes, edges_node_names_explainer_subgraphs], axis=1)

results_for_R.query("node1 == 'METLEUex'")

,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
577,"[6520.1, 8140.1]",METLEUex,leu_L_c,0.017892,0.018656,0.012535,0.046060,0.012905,0.025395,0.013120,...,0.023753,0.034653,0.046350,0.075669,0.035507,0.035372,0.008325,0.130646,0.025754,0.032150
578,"[6520.1, 8140.1]",METLEUex,leu_L_e,0.015301,0.018108,0.012095,0.039167,0.012481,0.036362,0.012547,...,0.043190,0.033277,0.034498,0.037429,0.032685,0.043219,0.018571,0.036918,0.040904,0.038387


In [6]:
results_for_R.to_csv("./results/data/Non_masked_results_for_R.csv")